<a href="https://colab.research.google.com/github/NathanielRose/eeg-294/blob/master/Gumpy-Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Testing for EEG | ML 294**

# Literature Details

- MI tasks across 2 sessions yielding a total of 750 trials.
- Each session consisted of 4 runs of 12 min separated by 10-min breaks to avoid mental fatigue. 
- Each run consisted of several MI tasks, each 10 s long. Att=4.5 s
- An arrow pointing either to the left or right was displayed with anacoustic warning tone (1 kHz, 70 ms)
- The subject was instructed to imagine a movement according tothe displayed cue (left vs. right) for four seconds.
- The MI task was followed by a relaxation period of1.5 s that separated two trials.


In [1]:
!git clone https://github.com/NathanielRose/eeg-294.git

Cloning into 'eeg-294'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 147 (delta 48), reused 84 (delta 11), pack-reused 0
Receiving objects: 100% (147/147), 1.42 MiB | 9.05 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [2]:
!pip install git+git://github.com/gumpy-bci/gumpy.git

  Cloning git://github.com/gumpy-bci/gumpy.git to /tmp/pip-req-build-cq_diako
  Running command git clone -q git://github.com/gumpy-bci/gumpy.git /tmp/pip-req-build-cq_diako
  Created wheel for gumpy: filename=gumpy-0.5.0-cp36-none-any.whl size=32999 sha256=de450aa8d43773c6d4ac45a23577d9b824deae8cd586d095e0315267035f7f19
  Stored in directory: /tmp/pip-ephem-wheel-cache-vnkuebmn/wheels/cf/ee/4d/dfa04a0d04ffb1ca36f436f3fef4817e9e23bfec03dd772913
Successfully built gumpy


In [3]:
##Import packages##

%reset
%matplotlib inline

from __future__ import print_function
import os; os.environ["THEANO_FLAGS"] = "device=gpu0"
import os.path
from datetime import datetime
import sys

import numpy as np
import scipy.io
import matplotlib.pyplot as plt
import pandas as pd

sys.path.append('gumpy/gumpy/')
import gumpy

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
sys.path.append('eeg-294/gumpy-deeplearning')
import models

In [0]:
sys.path.append('eeg-294/gumpy-deeplearning/models/')
import utils

In [0]:
DEBUG = True
CLASS_COUNT = 2
DROPOUT = 0.2   # dropout rate in float

# parameters for filtering data
FS = 250
LOWCUT = 2
HIGHCUT = 60
ANTI_DRIFT = 0.5
CUTOFF = 50.0 # freq to be removed from signal (Hz) for notch filter
Q = 30.0  # quality factor for notch filter 
W0 = CUTOFF/(FS/2)
AXIS = 0

#set random seed
SEED = 42
KFOLD = 5

# GET DATA


In [7]:
%%bash

export fileid=1DS64UOrt9wzswke5wiFtat94-s1YcRmb
export filename=grazdata.zip


wget --save-cookies cookies.txt 'https://docs.google.com/uc?export=download&id='$fileid -O- \
     | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1/p' > confirm.txt

wget --load-cookies cookies.txt -O $filename \
     'https://docs.google.com/uc?export=download&id='$fileid'&confirm='$(<confirm.txt)

--2019-11-15 02:59:58--  https://docs.google.com/uc?export=download&id=1DS64UOrt9wzswke5wiFtat94-s1YcRmb
Resolving docs.google.com (docs.google.com)... 74.125.195.101, 74.125.195.102, 74.125.195.100, ...
Connecting to docs.google.com (docs.google.com)|74.125.195.101|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘STDOUT’

     0K ...                                                    17.2M=0s

2019-11-15 02:59:59 (17.2 MB/s) - written to stdout [3248]

--2019-11-15 02:59:59--  https://docs.google.com/uc?export=download&id=1DS64UOrt9wzswke5wiFtat94-s1YcRmb&confirm=88ht
Resolving docs.google.com (docs.google.com)... 74.125.197.139, 74.125.197.113, 74.125.197.100, ...
Connecting to docs.google.com (docs.google.com)|74.125.197.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-94-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/8db4bnsupi599g2lfs

In [8]:
!unzip grazdata.zip

Archive:  grazdata.zip
mapname:  conversion of  failed
 extracting: B03T.mat                
 extracting: B05E.mat                
 extracting: B04T.mat                
 extracting: B01E.mat                
 extracting: B04E.mat                
 extracting: B06T.mat                
 extracting: B08T.mat                
 extracting: B07T.mat                
 extracting: B02T.mat                
 extracting: B03E.mat                
 extracting: B09T.mat                
 extracting: B05T.mat                
 extracting: B09E.mat                
 extracting: B01T.mat                
 extracting: B06E.mat                
 extracting: B07E.mat                
 extracting: B08E.mat                
 extracting: B02E.mat                


In [9]:
!unzip grazdata

Archive:  grazdata.zip
mapname:  conversion of  failed
replace B03T.mat? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
 extracting: B03T.mat                
 extracting: B05E.mat                
 extracting: B04T.mat                
 extracting: B01E.mat                
 extracting: B04E.mat                
 extracting: B06T.mat                
 extracting: B08T.mat                
 extracting: B07T.mat                
 extracting: B02T.mat                
 extracting: B03E.mat                
 extracting: B09T.mat                
 extracting: B05T.mat                
 extracting: B09E.mat                
 extracting: B01T.mat                
 extracting: B06E.mat                
 extracting: B07E.mat                
 extracting: B08E.mat                
 extracting: B02E.mat                


In [11]:
#load data
subject = 'B01'

grazb_data = gumpy.data.GrazB('', subject)
grazb_data.load()

In [12]:
grazb_data.print_stats()

Data identification: GrazB-B01
EEG-data shape: (1626266, 3)
Trials data shape:  (324,)
Labels shape:  (324,)
Total length of single trial:  8
Sampling frequency of EEG data: 250
Interval for motor imagery in trial:  [4, 7]
Classes possible:  [0 1]


# Postprocess data
Usually it is necessary to postprocess the raw data before you can properly use it. gumpy provides several methods to easily do so, or provides implementations that can be adapted to your needs.

Most methods internally use other Python toolkits, for instance sklearn, which is heavily used throughout gumpy. Thereby, it is easy to extend gumpy with custom filters. In addition, we expect users to have to manipulate the raw data directly as shown in the following example.

# Common average re-referencing the data to Cz
Some data is required to be re-referenced to a certain electrode. Because this may depend on your dataset, there is no common function provided by gumpy to do so. However and if sub-classed according to the documentation, you can access the raw-data directly as in the following example.

In [0]:
if 0:
    grazb_data.raw_data[:, 0] -= 2 * grazb_data.raw_data[:, 1]
    grazb_data.raw_data[:, 2] -= 2 * grazb_data.raw_data[:, 2]

# Example: Notch and Band-Pass Filters
gumpy ships with several filters already implemented. They accept either raw data to be filtered, or a subclass of Dataset. In the latter case, gumpy will automatically convert all channels using parameters extracted from the data

In [0]:
# this returns a butter-bandpass filtered version of the entire dataset
btr_data = gumpy.signal.butter_bandpass(grazb_data, lo=2, hi=60)

# it is also possible to use filters on individual electrodes using 
# the .raw_data field of a dataset. The example here will remove a certain
# from a single electrode using a Notch filter. This example also demonstrates
# that parameters will be forwarded to the internal call to the filter, in this
# case the scipy implementation iirnotch (Note that iirnotch is only available
# in recent versions of scipy, and thus disabled in this example by default)

# frequency to be removed from the signal
if False:
    f0 = 50.0 
    # quality factor
    Q = 50.0  
    # get the cutoff frequency
    w0 = f0/(grazb_data.sampling_freq/2) 
    # apply the notch filter
    btr_data = gumpy.signal.notch(btr_data.raw_data[:, 0], w0, Q)


# **Normalization**

Many datasets require normalization. gumpy provides functions to compute normalization either using a mean computation or via min/max computation. As with the filters, this function accepts either an instance of Dataset, or raw_data. In fact, it can be used for postprocessing any row-wise data in a numpy matrix.

In [15]:
# normalize the data first
norm_data = gumpy.signal.normalize(btr_data, 'mean_std')
# let's see some statistics
print("""Normalized Data:
  Mean    = {:.3f}
  Min     = {:.3f}
  Max     = {:.3f}
  Std.Dev = {:.3f}""".format(
  np.nanmean(norm_data),np.nanmin(norm_data),np.nanmax(norm_data),np.nanstd(norm_data)
))

Normalized Data:
  Mean    = 0.000
  Min     = -17.912
  Max     = 19.989
  Std.Dev = 1.000


In [17]:
!ls

B01E.mat  B03E.mat  B05E.mat  B07E.mat	B09E.mat     eeg-294
B01T.mat  B03T.mat  B05T.mat  B07T.mat	B09T.mat     grazdata.zip
B02E.mat  B04E.mat  B06E.mat  B08E.mat	confirm.txt  sample_data
B02T.mat  B04T.mat  B06T.mat  B08T.mat	cookies.txt


# Load Raw Data

In [18]:
sys.path.append('eeg-294/gumpy/gumpy')
# specify the location of the GrazB datasets
data_dir = '.'
subject = 'B01'

# initialize the data-structure, but do _not_ load the data yet
grazb_data = gumpy.data.GrazB(data_dir, subject)

# now that the dataset is setup, we can load the data. This will be handled from within the utils function, 
# which will first load the data and subsequently filter it using a notch and a bandpass filter.
# the utility function will then return the training data.
x_train, y_train = utils.load_preprocess_data(grazb_data, True, LOWCUT, HIGHCUT, W0, Q, ANTI_DRIFT, CLASS_COUNT, CUTOFF, AXIS, FS)

Band-pass filtering the data in frequency range from 2.0 Hz to 60.0 Hz... 
Data loaded and processed successfully!


# Augment Data

In [0]:
x_augmented, y_augmented = gumpy.signal.sliding_window(data = x_train[:,:,:],
                                                          labels = y_train[:,:],
                                                          window_sz = 4 * FS,
                                                          n_hop = FS // 10,
                                                          n_start = FS * 1)
x_subject = x_augmented
y_subject = y_augmented
x_subject = np.rollaxis(x_subject, 2, 1)


# Run the model

In [0]:

from sklearn.model_selection import StratifiedKFold
from models import CNN_STFT

# define KFOLD-fold cross validation test harness
kfold = StratifiedKFold(n_splits = KFOLD, shuffle = True, random_state = SEED)
cvscores = []
ii = 1
for train, test in kfold.split(x_subject, y_subject[:, 0]):
    print('Run ' + str(ii) + '...')
    # create callbacks
    model_name_str = 'GRAZ_CNN_STFT_3layer_' + \
                     '_run_' + str(ii)
    #callbacks_list = model.get_callbacks(model_name_str)

    # initialize and create the model
    model = CNN_STFT(model_name_str)
    model.create_model(x_subject.shape[1:], dropout = DROPOUT, print_summary = False)
    
    # fit model. If you specify monitor=True, then the model will create callbacks
    # and write its state to a HDF5 file
    model.fit(x_subject[train], y_subject[train], monitor=True,
              epochs = 100, 
              batch_size = 256, 
              verbose = 0, 
              validation_split = 0.1)
    #          validation_split = 0.1, callbacks = callbacks_list)
    # They removed the callbacks from the script for tensorboard evaluation
    

    # evaluate the model
    print('Evaluating model on test set...')
    scores = model.evaluate(x_subject[test], y_subject[test], verbose = 0)

    print("Result on test set: %s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))
    cvscores.append(scores[1] * 100)
    ii += 1
    
# print some evaluation statistics and write results to file
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
cv_all_subjects = np.asarray(cvscores)
print('Saving CV values to file....')
np.savetxt('GRAZ_CV_' + 'CNN_STFT_3layer_' + str(DROPOUT) + 'do'+'.csv', 
            cv_all_subjects, delimiter = ',', fmt = '%2.4f')
print('CV values successfully saved!\n')

Run 1...


eeg-294/gumpy-deeplearning/models/cnn_stft.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(filters=24, kernel_size=(12, 12), strides=(1, 1), name="conv1", padding="same")`
  border_mode = 'same'))
eeg-294/gumpy-deeplearning/models/cnn_stft.py:36: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(filters=48, kernel_size=(8, 8), name="conv2", padding="same")`
  name = 'conv2', border_mode = 'same'))
eeg-294/gumpy-deeplearning/models/cnn_stft.py:44: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(filters=96, kernel_size=(4, 4), name="conv3", padding="same")`
  name = 'conv3', border_mode = 'same'))


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


